In [1]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 18.4 MB/s eta 0:00:00


In [2]:
!pip install --upgrade pytorch-lightning

In [3]:
!pip install transformers

In [4]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from pathlib import Path
import pytorch_lightning as pl

In [5]:
from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap

from torch.utils.data import Dataset, DataLoader
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint

from lightning.pytorch.loggers import TensorBoardLogger
from transformers import AdamW, T5ForConditionalGeneration, T5TokenizerFast as T5Tokenizer
from tqdm.auto import tqdm

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
pl.seed_everything(1234)

INFO:lightning_fabric.utilities.seed:Seed set to 1234


1234

In [8]:
df = pd.read_csv("data.csv", encoding = "utf-8")
df.head()

,Title,Content,Summary
0,Đề xuất bổ sung gần 28.000 biên chế giáo viên,"Theo Bộ Nội vụ, năm học này, các địa phương đề...",Bộ Nội vụ cùng Bộ Giáo dục và Đào tạo ...
1,Lộ trình du học Australia với 700 triệu đồng,"Theo đó, Tổ chức Giáo dục QTS Australia sẽ cùn...",QTS Australia áp dụng chương trình hỗ trợ tài ...
2,Dự kiến cho học sinh phổ thông học trước chươn...,"Thông tin được PGS Vũ Hải Quân, Giám đốc Đại h...",Học sinh giỏi vượt trội ở bậc THPT có thể được...
3,Dự kiến thêm nhiều chứng chỉ được miễn thi tốt...,"Trong dự thảo sửa đổi, bổ sung quy chế thi tốt...",Bộ Giáo dục và Đào tạo dự kiến miễn thi và tín...
4,Thu hàng trăm nghìn USD để tư vấn vào Ivy League,"Rim, người sáng lập Command Education, tổ chức...","MỹChristopher Rim, 28 tuổi, ra giá 750.000 USD..."


In [9]:
df = df[["Content", "Summary"]]
df.columns = ["text", "summary"]

In [10]:
print(df.isnull().sum())

text       0
summary    0
dtype: int64


In [11]:
df = df.dropna()
df.head()

,text,summary
0,"Theo Bộ Nội vụ, năm học này, các địa phương đề...",Bộ Nội vụ cùng Bộ Giáo dục và Đào tạo ...
1,"Theo đó, Tổ chức Giáo dục QTS Australia sẽ cùn...",QTS Australia áp dụng chương trình hỗ trợ tài ...
2,"Thông tin được PGS Vũ Hải Quân, Giám đốc Đại h...",Học sinh giỏi vượt trội ở bậc THPT có thể được...
3,"Trong dự thảo sửa đổi, bổ sung quy chế thi tốt...",Bộ Giáo dục và Đào tạo dự kiến miễn thi và tín...
4,"Rim, người sáng lập Command Education, tổ chức...","MỹChristopher Rim, 28 tuổi, ra giá 750.000 USD..."


In [12]:
df.shape

(84, 2)

In [13]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

In [14]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set(style='whitegrid', palette='muted', font_scale = 1.2)
rcParams['figure.figsize'] = 16,10

In [15]:
train_df, test_df = train_test_split(df, test_size=0.1)
print(f"Shape of the Train Set: {train_df.shape}\nShape of the Test Set: {test_df.shape}")

Shape of the Train Set: (75, 2)
Shape of the Test Set: (9, 2)


In [16]:
class NewsDataset(Dataset):
    def __init__(self, data, tokenizer, text_max_token_len=512, summary_max_token_len=128):
        """
        A dataset that represents news articles and their respective summaries.

        Args:
        - data (pd.DataFrame): The data that contains the news articles and their summaries.
        - tokenizer (transformers.tokenization_*) : The tokenizer used to tokenize the text and summary.
        - text_max_token_len (int, optional): The maximum length of the text in terms of tokens. Defaults to 512.
        - summary_max_token_len (int, optional): The maximum length of the summary in terms of tokens. Defaults to 128.
        """
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len

    def __len__(self):
        """
        Returns:
        - The number of samples in the dataset.
        """
        return len(self.data)

    def __getitem__(self, index):
        """
        Get a sample from the dataset.

        Args:
        - index (int): The index of the sample to get.

        Returns:
        - A dictionary that contains the following:
            - text (str): The original text of the news article.
            - summary (str): The summary of the news article.
            - text_input_ids (torch.Tensor): The input IDs of the text after tokenization.
            - text_attention_mask (torch.Tensor): The attention mask of the text after tokenization.
            - labels (torch.Tensor): The input IDs of the summary after tokenization.
            - labels_attention_mask (torch.Tensor): The attention mask of the summary after tokenization.
        """
        data_row = self.data.iloc[index]
        text = data_row["text"]

        # Encode the text
        text_encoding = tokenizer(
            text,
            max_length=self.text_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        # Encode the summary
        summary_encoding = self.tokenizer(
            data_row["summary"],
            max_length=self.summary_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

        # Modify the labels so that the model knows which tokens to predict
        labels = summary_encoding['input_ids']
        labels[labels == 0] = -100

        return {
            'text': text,
            'summary': data_row['summary'],
            'text_input_ids': text_encoding['input_ids'].flatten(),
            'text_attention_mask': text_encoding['attention_mask'].flatten(),
            'labels': labels.flatten(),
            'labels_attention_mask': summary_encoding["attention_mask"].flatten()
        }

In [17]:
class NewsDataModule(pl.LightningDataModule):
    def __init__(self,
                 train_df,
                 test_df,
                 tokenizer,
                 batch_size=8,
                 text_max_token_len=512,
                 summary_max_token_len=128):
        """
        Initializes the NewsDataModule.

        Args:
        - train_df (pandas.DataFrame): the training dataset
        - test_df (pandas.DataFrame): the testing dataset
        - tokenizer (transformers.PreTrainedTokenizer): the tokenizer to be used
        - batch_size (int): the batch size
        - text_max_token_len (int): the maximum number of tokens for the text
        - summary_max_token_len (int): the maximum number of tokens for the summary
        """
        super().__init__()

        self.train_df = train_df
        self.test_df = test_df

        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len

    def setup(self, stage=None):
        """
        Sets up the dataset.
        """
        self.train_dataset = NewsDataset(
            self.train_df,
            self.tokenizer,
            self.text_max_token_len,
            self.summary_max_token_len)

        self.test_dataset = NewsDataset(
            self.test_df,
            self.tokenizer,
            self.text_max_token_len,
            self.summary_max_token_len)

    def train_dataloader(self):
        """
        Returns the DataLoader for the training set.
        """
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True
        )

    def test_dataloader(self):
        """
        Returns the DataLoader for the testing set.
        """
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False
        )

    def val_dataloader(self):
        """
        Returns the DataLoader for the validation set, which is the same as the testing set.
        """
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False
        )

In [18]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = "VietAI/vit5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

In [19]:
text_token_counts = [len(tokenizer.encode(row["text"])) for _, row in train_df.iterrows()]
summary_token_counts = [len(tokenizer.encode(row["summary"])) for _, row in train_df.iterrows()]

In [20]:
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
# sns.histplot(text_token_counts, ax=ax1, color='blue', alpha=0.7)
# ax1.set_title("Distribution of Text Token Counts", fontsize=14, fontweight='bold')
# ax1.set_xlabel("Number of Tokens", fontsize=12)
# ax1.set_ylabel("Frequency", fontsize=12)
# ax1.grid(axis='y', alpha=0.5)

# sns.histplot(summary_token_counts, ax=ax2, color='green', alpha=0.7)
# ax2.set_title("Distribution of Summary Token Counts", fontsize=14, fontweight='bold')
# ax2.set_xlabel("Number of Tokens", fontsize=12)
# ax2.set_ylabel("Frequency", fontsize=12)
# ax2.grid(axis='y', alpha=0.5)

# plt.suptitle("Token Count Distributions", fontsize=16, fontweight='bold')
# plt.tight_layout(rect=[0, 0.03, 1, 0.95])
# plt.show()

In [21]:
N_EPOCHS = 5
BATCH_SIZE = 8

data_module = NewsDataModule(
    train_df,
    test_df,
    tokenizer,
    batch_size=BATCH_SIZE

)
data_module.setup()

Model

In [22]:
from transformers import BertForSequenceClassification, AdamW
import pytorch_lightning as pl

class SummaryModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, return_dict=True)

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        return output.loss, output.logits

    def shared_step(self, batch, batch_idx, stage):
        input_ids = batch['text_input_ids']
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]

        loss, _ = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        self.log(f"{stage}_loss", loss, prog_bar=True, logger=True)
        return loss

    def training_step(self, batch, batch_idx):
        return self.shared_step(batch, batch_idx, 'train')

    def validation_step(self, batch, batch_idx):
        return self.shared_step(batch, batch_idx, 'val')

    def test_step(self, batch, batch_idx):
        return self.shared_step(batch, batch_idx, 'test')

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=0.0001)

In [23]:
model = SummaryModel()

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

In [24]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

callbacks = ModelCheckpoint(
    dirpath="/content/MyDrive/",
    filename="base-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode='min'
)

logger = TensorBoardLogger("lightning_logs", name="news_summary")


trainer = Trainer(
    logger=logger,
    callbacks=callbacks,
    max_epochs=N_EPOCHS,
    accelerator='gpu'
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [25]:
# import torch
# torch.cuda.empty_cache()

In [26]:
trainer.fit(model, data_module)
# trainer.fit(model, train_dataloaders=data_module.train_dataloader(), val_dataloaders=data_module.val_dataloader())
# trainer.fit(model, train_dataloader=data_module.train_dataloader())
# trainer.fit(model, train_dataloader=data_module.train_dataloader(), val_dataloaders=data_module.val_dataloader())

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 225 M 
-----------------------------------------------------
225 M     Trainable params
0         Non-trainable params
225 M     Total params
903.804   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO: Epoch 2, global step 30: 'val_loss' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 2, global step 30: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 3, global step 40: 'val_loss' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 3, global step 40: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 4, global step 50: 'val_loss' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 4, global step 50: 'val_loss' was not in top 1
INFO: `Trainer.fit` stopped: `max_epochs=5` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [27]:
best_model = SummaryModel.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path
)
best_model.freeze()

In [28]:
# def summarizeText(text):
#     text_encoding = tokenizer(
#         text,
#         max_length=512,
#         padding='max_length',
#         truncation=True,
#         return_attention_mask=True,
#         add_special_tokens=True,
#         return_tensors='pt'
#     )
#     generated_ids = model.generate(
#         input_ids=text_encoding['input_ids'],
#         attention_mask=text_encoding['attention_mask'],
#         max_length=150,
#         num_beams=2,
#         repetition_penalty=2.5,
#         length_penalty=1.0,
#         early_stopping=True
#     )

#     preds = [
#             tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
#             for gen_id in generated_ids
#     ]
#     return "".join(preds)

In [29]:
def encode_text(text):
    # Encode the text using the tokenizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    encoding = tokenizer.encode_plus(
        text,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
#     return encoding["input_ids"], encoding["attention_mask"]
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)
    return input_ids, attention_mask

def generate_summary(input_ids, attention_mask):
    # Generate a summary using the best model
    generated_ids = best_model.model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=150,
        num_beams=2,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )
    return generated_ids

def decode_summary(generated_ids):
    # Decode the generated summary
    summary = [tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
               for gen_id in generated_ids]
    return "".join(summary)

def summarize(text):
    input_ids, attention_mask = encode_text(text)
    generated_ids = generate_summary(input_ids, attention_mask)
    summary = decode_summary(generated_ids)
    return summary


In [30]:
sample_row = test_df.iloc[2]
text = sample_row["text"]
model_summary = summarize(text)

print("News: ", text)
print("\nOriginal Summary: ", sample_row["summary"])
print("\nPredicted Summary: ", model_summary)

from nltk.translate.bleu_score import sentence_bleu

reference = [sample_row["summary"]]
candidate = model_summary

bleu_score = sentence_bleu(reference, candidate)

print("\nBLEU score:", bleu_score)

News:  Theo cô Lê Thị Bích Dung - Phó Chủ tịch Hội đồng quản trị Hệ thống Trường liên cấp Newton tinh thần giáo dục của trường được đúc kết trong slogan: "Mỗi học sinh là một nhà lãnh đạo tương lai". Để hiện thực hóa triết lý này, nhà trường triển khai phương thức học trải nghiệm thú vị. Theo phương pháp giáo dục truyền thống, người học sẽ tiếp thu kiến thức bằng cách đọc chép, nghe nhìn, trình chiếu. Với phương pháp dạy học trải nghiệm, học sinh được tiếp thu kiến thức thông qua thực hành, các dự án, STEM, tham gia cuộc thi, hoạt động dã ngoại, trại hè trong nước và quốc tế.Thông qua các hoạt động trải nghiệm, học sinh sẽ có nền tảng tri thức vững chắc, tăng sự hứng thú, tính chủ động, và thúc đẩy tiềm năng sáng tạo của bản thân. Học sinh có cơ hội tiếp cận tri thức bằng nhiều giác quan khác nhau như thị giác, thính giác, xúc giác, khứu giác... Điều này góp phần tăng khả năng ghi nhớ cho học sinh, tăng kỹ năng cần thiết như kỹ năng tổng hợp thông tin, phân tích, kỹ năng giao tiếp, sán

In [31]:
sample_row = test_df.iloc[3]
text = sample_row["text"]
model_summary = summarize(text)

print("News: ", text)
print("\nOriginal Summary: ", sample_row["summary"])
print("\nPredicted Summary: ", model_summary)

from nltk.translate.bleu_score import sentence_bleu

reference = [sample_row["summary"]]
candidate = model_summary

bleu_score = sentence_bleu(reference, candidate)

print("\nBLEU score:", bleu_score)

News:  Tại buổi làm việc giữa Bộ trưởng Giáo dục và Đào tạo Nguyễn Kim Sơn với Học viện Kỹ thuật Quân sự chiều 13/12, Thượng tướng Lê Huy Vịnh, Thứ trưởng Bộ Quốc phòng, đề nghị Bộ Giáo dục và Đào tạo ủng hộ Bộ Quốc phòng tham gia đào tạo nguồn nhân lực phục vụ công nghiệp hóa, hiện đại hóa đất nước, thay vì chỉ phục vụ riêng quân đội như hiện nay.Từ năm 2002, các trường quân đội đã tuyển sinh được hơn 93.000 sinh viên hệ dân sự ở các trình độ. Đến năm 2017, theo nghị quyết 19, Hội nghị Ban chấp hành Trung ương Đảng khóa XII, các trường quân đội từng bước giảm dần và đến năm 2021 dừng hẳn tuyển sinh hệ dân sự. Việc này nhằm thực hiện chủ trương đổi mới, sắp xếp tổ chức các đơn vị quân đội, khắc phục tình trạng có trường đào tạo hệ dân sự gấp nhiều lần hệ quân sự, nhưng đội ngũ cán bộ giảng dạy vẫn là của quân đội.Tuy nhiên, trước tình hình khó khăn về nguồn nhân lực chất lượng cao, trong khi các trường quân đội đủ điều kiện và có thế mạnh ở nhiều ngành, Bộ Quốc phòng đang hoàn thiện đề

In [32]:
from nltk.translate.bleu_score import corpus_bleu

references = []
candidates = []

for i in range(len(test_df)):
    sample_row = test_df.iloc[i]
    text = sample_row["text"]
    model_summary = summarize(text)

    references.append([sample_row["summary"]])
    candidates.append(model_summary)

bleu_score = corpus_bleu(references, candidates)

print("BLEU score:", bleu_score)

BLEU score: 0.348768897903224


In [33]:
model_summary = summarize("""Thí sinh dự thi vào lớp 10 chuyên chỉ được đăng ký tối đa 5 nguyện vọng, thay vì 7 như trước đây, theo dự kiến của Sở Giáo dục và Đào tạo.
Lý giải về điều chỉnh này, đại diện Sở ngày 5/1 cho biết từ năm học 2024-2025, TP HCM dừng tuyển sinh lớp không chuyên trong các trường chuyên theo quy định của Bộ Giáo dục và Đào tạo. Do vậy, Sở điều chỉnh số lượng và cách thức đăng ký nguyện vọng để phù hợp với thực tế.
Cụ thể, thí sinh thi lớp 10 năm nay vẫn được đăng ký tối đa ba nguyện vọng vào các trường THPT công lập dạy chương trình đại trà (nguyện vọng thường). Việc xét tuyển căn cứ vào thứ tự đăng ký nguyện vọng của các em, trong đó điểm chuẩn nguyện vọng 2 không thấp hơn nguyện vọng 1, nguyện vọng 3 không thấp hơn nguyện vọng 2.
Thí sinh muốn thi trường chuyên sẽ được đăng ký thêm hai nguyện vọng, thay vì bốn như trước đây. Ở nguyện vọng chuyên 1, thí sinh có thể đăng ký vào trường THPT chuyên Lê Hồng Phong và Trần Đại Nghĩa. Với nguyện vọng chuyên 2, ngoài hai trường này, thí sinh có thể đăng ký vào lớp chuyên của bốn trường khác gồm: THPT Nguyễn Hữu Huân, Nguyễn Thượng Hiền, Mạc Đĩnh Chi và Gia Định.
Nếu thi vào lớp 10 tích hợp, ngoài ba nguyện vọng thường, thí sinh có thêm hai nguyện vọng tích hợp. Thí sinh đăng ký dự thi chuyên sẽ không được thi tích hợp và ngược lại.
Như vậy, một thí sinh có thể đăng ký tối đa 5 nguyện vọng.
Năm nay, TP HCM tiếp tục cho học sinh đăng ký nguyện vọng thi lớp 10 bằng hình thức trực tuyến. Sở lưu ý học sinh chọn trường gần nơi cư trú, tránh trường hợp trúng tuyển nhưng không nhập học vì nhà xa. Học sinh không được thay đổi nguyện vọng sau thời gian điều chỉnh và sau khi có kết quả trúng tuyển.
Kỳ thi vào lớp 10 công lập năm 2024 ở TP HCM dự kiến diễn ra vào đầu tháng 6 với ba môn thi bắt buộc. Trong đó, môn Toán và Ngữ văn kéo dài 120 phút, Ngoại ngữ 90 phút. Thí sinh dự thi vào lớp 10 chuyên, tích hợp sẽ dự thi thêm môn chuyên, tích hợp với thời gian là 150 phút. Sở Giáo dục và Đào tạo cho biết cấu trúc đề thi tương tự mọi năm, đề cao khả năng giải quyết vấn đề thực tế của thí sinh.
Từ tháng 1 đến tháng 4, Sở triển khai tư vấn tuyển sinh vào lớp 10 cho học sinh lớp 9 tại nơi các em đang học, với sự tham gia của các trường THPT trong và ngoài công lập, chuyên viên Sở, chuyên gia tư vấn tâm lý.
Ngoài ra, Sở thay đổi toàn bộ quy trình xét nguyện vọng thi lớp 10 công lập, tránh tình trạng nhiều trường tuyển không đủ chỉ tiêu, trong khi có thí sinh điểm cao lại trượt hết nguyện vọng. Phương án cụ thể sẽ được công bố sau.""")

model_summary

'Bộ Giáo dục và Đào tạo điều chỉnh số nguyện vọng dự thi vào lớp 10 công lập để phù hợp với quy định của Bộ Giáo dục và Đào tạo.'

In [34]:
model_summary = summarize("""Hơn 5.800 thí sinh tham dự kỳ thi chọn học sinh giỏi quốc gia năm nay, tăng 1.230 người so với năm ngoái.
Các môn thi gồm Toán, Vật lý, Hóa học, Sinh học, Tin học, Ngữ văn, Lịch sử, Địa lý, Tiếng Anh, Tiếng Pháp, Tiếng Nga và Tiếng Trung.
Theo thống kê ngày 4/1 của Bộ Giáo dục và Đào tạo, Ngữ văn là môn có nhiều thí sinh dự thi nhất - gần 650, kế đó là Tiếng Anh 639. Hai môn Toán và Sinh học cùng có 607 thí sinh, Vật lý 605.
Môn ít thí sinh nhất là Tiếng Nga (77 thí sinh) và môn Tiếng Trung (104 thí sinh).
Tổng thí sinh tham gia kỳ thi chọn học sinh giỏi năm nay là 5.819, tăng 1.230 so với năm ngoái.
Kỳ thi chọn học sinh giỏi quốc gia diễn ra trong hai ngày 5-6/1, dành cho học sinh THPT. Nội dung thi thuộc chương trình giáo dục phổ thông 2006. Trừ Ngữ văn, Lịch sử và Địa lý, các môn còn lại đều diễn ra trong hai ngày.
Hồi tháng 10/2023, Bộ Giáo dục và Đào tạo ban hành quy chế thi học sinh giỏi quốc gia với nhiều thay đổi so với quy định được áp dụng từ năm 2011. Trong đó, điểm nổi bật là tăng số thí sinh dự thi và tỷ lệ đạt giải.
Trước đây, Bộ quy định đội tuyển mỗi môn thi của từng đơn vị nhiều nhất có 6 thí sinh. Riêng Hà Nội, đội tuyển mỗi môn được tối đa 12 thí sinh. Theo quy định mới, số thí sinh của các đơn vị ở mỗi môn tối đa là 10, riêng TP HCM và Hà Nội là 20. Đây là lý do khiến số lượng thí sinh dự thi năm nay tăng mạnh so với năm ngoái.
Tỷ lệ học sinh đạt giải sẽ chiếm 60% tổng số thí sinh, tăng 10% so với trước. Trong đó, số giải nhất, nhì, ba không vượt quá 60% tổng số giải; số giải nhất không vượt quá 5%. Tỷ lệ này, theo Bộ là phù hợp với quy định của các kỳ thi Olympic khu vực và quốc tế.
Những thí sinh tham gia nhưng không đạt giải sẽ được cấp giấy chứng nhận để các em có thông tin lưu giữ lâu dài. Đây là điểm khác so với trước.
Bộ Giáo dục và Đào tạo cho biết kỳ thi chọn học sinh giỏi quốc gia nhằm động viên, khuyến khích người dạy và học phát huy năng lực sáng tạo, góp phần thúc đẩy và cải thiện chất lượng giáo dục. Đồng thời, việc này để phát hiện học sinh có năng khiếu, từ đó tạo nguồn bồi dưỡng, thực hiện mục tiêu đào tạo nhân tài cho đất nước.
Năm 2023, hơn 4.500 thí sinh dự thi học sinh giỏi quốc gia ở 12 môn. Hơn 2.200 thí sinh đạt giải, Hà Nội dẫn đầu (141 em), kế đến là Nghệ An (87), Vĩnh Phúc (79), Hải Phòng (76).
Học sinh đạt giải được ưu tiên xét tuyển thẳng vào đại học, cao đẳng. Những thí sinh được vào vòng chọn đội tuyển Olympic quốc tế (Tin học, Hóa học, Toán học, Sinh học, Vật lý) sẽ được miễn thi tốt nghiệp THPT.""")

model_summary

'Bộ Giáo dục và Đào tạo vừa ban hành quy chế thi học sinh giỏi quốc gia, tăng 10% so với năm ngoái.'